In [1]:
%load_ext autoreload
%autoreload 2
import torch, rdkit
import sys, pathlib
from pathlib import Path
PROJECT_ROOT = Path.home()/"바탕화면"/"torch"/"Chem"
sys.path.insert(0, str(PROJECT_ROOT))
from rdkit import Chem
from utils.utils import *


from tqdm import trange
from pathlib import Path
device   = "cuda"

vocab = {'(': 0, ')': 1, '=': 2, '[O;!R;C]': 3, '[CH3;!R;C]': 4, '[CH2;!R;CN]': 5, '[CH2;!R;CC]': 6, '[CH2;!R;CO]': 7, '[*;!R;C]': 8, '[SOS]': 9, '[EOS]': 10, '[O;!R;CC]': 11, '[NH;!R;CC]': 12, '[*;!R;O]': 13, '[O;!R;*C]': 14, '[C;!R;*OO]': 15, '[CH;!R;CCO]': 16, '[CH;!R;CCN]': 17, '[C;!R;CNO]': 18, '[N;!R;CCC]': 19, '[C;!R;*NO]': 20, '[NH;!R;*C]': 21, '[*;!R;N]': 22, '[CH3;!R;N]': 23, '[CH;!R;CCC]': 24, '[C;!R;CCCO]': 25, '[CH;!R;CC]': 26, '[F;!R;C]': 27, '[CH2;!R;C]': 28, '[C;!R;COO]': 29, '[CH3;!R;O]': 30, '[C;!R;CCCN]': 31, '#': 32, '[C;!R;CCCC]': 33, '[CH2;!R;CS]': 34, '[C;!R;CC]': 35, '[OH;!R;C]': 36, '[S;!R;CC]': 37, '[N;!R;C]': 38, '[C;!R;CN]': 39, '[CH;!R;C]': 40, '[C;!R;CCC]': 41, '[NH2;!R;C]': 42, '[CH;!R;CFF]': 43, '[O;!R;S]': 44, '[O;!R;CN]': 45, '[CH;!R;CCS]': 46, '[CH2;!R;CF]': 47, '[CH3;!R;S]': 48, '[C;!R;CCO]': 49, '[NH;!R;CO]': 50, '[CH2;!R;*C]': 51, '[C;!R;NNO]': 52, '[C;!R;CFFF]': 53, '[C;!R;CCFF]': 54, '[C;!R;NOO]': 55, '[C;!R;OOO]': 56, '[S;!R;CCOO]': 57, '[CH3;!R;Si]': 58, '[C;!R;NNS]': 59, '[S;!R;C]': 60, '[NH;!R;C]': 61, '[Cl;!R;C]': 62, '[OH;!R;P]': 63, '1': 64, '[C;!R;CNN]': 65, '[CH;!R;CN]': 66, '[SH;!R;C]': 67, '[CH;!R;CCF]': 68, '[O;!R;P]': 69, '[CH2;!R;NO]': 70, '[O;R;CC]': 71, '[C;!R;CCCl]': 72, '[CH2;!R;CSi]': 73, '[CH;!R;NN]': 74, '[cH;R;CC]': 75, '[c;R;CCO]': 76, '[CH;!R;CO]': 77, '[Si;!R;CCCC]': 78, '[C;!R;CCN]': 79, '[N;!R;CN]': 80, '[NH;!R;CS]': 81, '[CH;!R;COO]': 82, '[N;!R;CCO]': 83, '[P;!R;COOO]': 84, '[C;!R;CCNN]': 85, '[N;!R;CO]': 86, '[CH;!R;CCl]': 87, '[S;!R;CNOO]': 88, '[C;!R;CCCF]': 89, '[CH2;!R;NSi]': 90, '[O;!R;CS]': 91, '[NH2;!R;O]': 92, '[NH;!R;CN]': 93, '[CH;R;CCO]': 94, '[CH2;R;CO]': 95, '[C;R;OOO]': 96, '[C;!R;CCF]': 97, '[C;!R;CCOO]': 98, '[N;!R;CCN]': 99, '[N;!R;CS]': 100, '[S;!R;CCNO]': 101, '[OH;!R;N]': 102, '[C;!R;NNN]': 103, '[S;!R;CCO]': 104, '[CH2;!R;CP]': 105, '[CH;!R;CNP]': 106, '[OH;!R;S]': 107, '[NH2;!R;N]': 108, '[N;!R;CC]': 109, '[Si;!R;*CCC]': 110, '[*;!R;Si]': 111, '[CH;!R;CS]': 112, '[C;!R;*CO]': 113, '[S;!R;COOO]': 114, '[O;!R;CP]': 115, '[S;!R;NNOO]': 116, '[NH2;!R;S]': 117, '[CH;R;*CN]': 118, '[N;R;CCC]': 119, '2': 120, '[CH2;!R;OO]': 121, '[C;!R;NN]': 122, 'N': 123, '[S;!R;COOS]': 124, '[S;!R;S]': 125, '[C;!R;CCCS]': 126, '[CH2;!R;NS]': 127, '[P;!R;OOOO]': 128, '[CH;!R;CNO]': 129, '[S;!R;OOO]': 130, '[O;!R;*S]': 131, '[CH2;!R;SS]': 132, '[CH;!R;FFO]': 133, '[CH2;!R;OP]': 134, '[C;!R;NSS]': 135, '[CH;!R;COP]': 136, '[C;!R;CFFO]': 137, '[CH;!R;CNS]': 138, '[c;R;CCC]': 139, '[CH;!R;CNN]': 140, '[C;!R;CSi]': 141, '[C;R;NNO]': 142, '[c;R;*CC]': 143, '[CH;!R;*CC]': 144, '[CH2;!R;NN]': 145, '[[N+]': 146, 'O': 147, '[[O-]': 148, '[N;!R;NO]': 149, '[CH;!R;NO]': 150, '[CH2;!R;OSi]': 151, '[CH;!R;NNN]': 152, '[P;!R;CCOO]': 153, '[C;!R;CClCl]': 154, '[C;!R;CCNO]': 155, '[C;!R;COS]': 156, '[N;!R;CCS]': 157, '[C;!R;CCCCl]': 158, '[C;!R;CCFO]': 159, '[C;!R;CCOP]': 160, '[SH;!R;O]': 161, '[S;!R;OOOO]': 162, '[C;!R;OSS]': 163, '[C;!R;CCClCl]': 164, '[PH;!R;CCO]': 165, '[PAD]': 166}
index_to_token = {idx: token for token, idx in vocab.items()}

cuda


In [2]:
def select_model(choice):
    if choice == "Trans_MHA":
        from models.Trans_MHA import CVAE, PriorNet
        model    = CVAE().cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=64).cuda()

    # Trans
    elif choice == "Trans":
        from models.Trans import CVAE, PriorNet
        model    = CVAE().cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=64).cuda()

    # LSTM
    elif choice == "LSTM":
        from models.LSTM import CVAE, PriorNet
        model    = CVAE().cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=64).cuda()

    # LSTM + MHA
    elif choice == "LSTM_MHA":
        from models.LSTM_MHA import CVAE, PriorNet
        model    = CVAE().cuda()
        model.decoder.cuda()
        prior = PriorNet(y_dim=3, latent_dim=64).cuda()

    return model, prior

def get_loss_fn(choice):
    if choice == "Trans_MHA":
        loss_fn = ConditionalVAELoss(
            vocab_size=dataset.vocab_size,
            max_beta=2.0,
            anneal_steps=2400,
            free_bits=0.3,
            capacity_max=0.0,
            capacity_inc=0.0,
            gamma=0.0,
            prop_w=3.0,
            nce=0.3,
            sig_pen_p=0.1,
            sig_pen_q=0.0,
            imb=0.1
        ).cuda()

    # Trans
    elif choice == "Trans":
        loss_fn = ConditionalVAELoss(
            vocab_size=dataset.vocab_size,
            max_beta=0.5,
            anneal_steps=2400,
            free_bits=0.03,
            capacity_max=0.0,
            capacity_inc=0.0,
            gamma=0.0,
            prop_w=3.0,
            nce=0.03,
            sig_pen_p=3,
            sig_pen_q=0.0,
            imb=0.05
        ).cuda()

    # LSTM
    elif choice == "LSTM":
        loss_fn = ConditionalVAELoss_LSTM(
            vocab_size=dataset.vocab_size,
            max_beta=0.08,
            anneal_steps=2400,
            free_bits=0.03,
            capacity_max=0.0,
            capacity_inc=0.0,
            gamma=0.0,
            prop_w=3.0,
            nce=0.2,
            sig_pen_p=0.05,
            sig_pen_q=0.1,
            imb=0.5
        ).cuda()

    # LSTM + MHA
    elif choice == "LSTM_MHA":
        loss_fn = ConditionalVAELoss(
            vocab_size=dataset.vocab_size,
            max_beta=1.0,
            anneal_steps=2400,
            free_bits=0.03,
            capacity_max=0.0,
            capacity_inc=0.0,
            gamma=0.0,
            prop_w=3.0,
            nce=0.2,
            sig_pen_p=0.3,
            sig_pen_q=0.0,
            imb=0.5
        ).cuda()
        
    return loss_fn

In [3]:
mode = "Trans_MHA"
model, prior = select_model(mode)
loss_fn = get_loss_fn(mode)

lr = 3e-5
lr_prior = 1e-4
from torch.optim import AdamW
optim = AdamW(model.parameters(), lr=lr)
optim2 = AdamW(prior.parameters(), lr=lr_prior)

In [ ]:
import datetime
status_out = widgets.Output()
display(status_out)
epoch = 1200
model.train()
prior.train()
progress = tqdm(range(epoch), desc="Training")
loss_arr=[]
for i in progress:
    log_var_extract=[]
    log_var_p_extract = []
    batchloss = 0.0
    embeddings = []
    mean_extract = []
    kld_raw_batch = 0.0
    bce = 0.0
    kld = 0.0
    if mode=="Trans" or mode=="Trans_MHA":
      for (smiles_enc, smiles_dec_input, smiles_dec_output, properties) in train_dataloader:
        optim.zero_grad()
        optim2.zero_grad()

        smiles_enc = smiles_enc.to(device)
        smiles_dec_input = smiles_dec_input.to(device)
        smiles_dec_output = smiles_dec_output.to(device)
        properties = properties.to(device)
        output, tgt, means, log_var, tgt_z = model.forward(smiles_enc, smiles_dec_input, properties)
      
        mu_p, logvar_p = prior.forward(properties.squeeze())
        with torch.no_grad():
          log_var_extract.append(torch.exp(0.5 * log_var.cpu()).to(device))
          log_var_p_extract.append(torch.exp(0.5 * logvar_p.cpu()).to(device))
          mean_extract.append(means.cpu())
        
        loss, BCE, KLD, prop, kld_raw = loss_fn.forward(output.float(),
                                smiles_dec_output,
                                means, log_var, mu_p, logvar_p,
                                tgt, properties.float().squeeze(), tgt_z,
                                i)

        loss.backward()
        optim.step()
        optim2.step()

        batchloss += loss.item()
        kld_raw_batch += kld_raw.item()
        bce += BCE.item()
        kld += KLD.item()
    else:
      for (smiles_dec_input, smiles_dec_output, properties) in train_dataloader_LSTM:
        tf_ratio = max(0.1, 1.0 - i / 300)
        optim.zero_grad()
        optim2.zero_grad()

        smiles_dec_input = smiles_dec_input.to(device)
        smiles_dec_output = smiles_dec_output.to(device)
        properties = properties.to(device)

        output, tgt, means, log_var, tgt_z = model.forward(smiles_dec_input, smiles_dec_output, properties, tf_ratio)
        
        mu_p, logvar_p = prior.forward(properties.squeeze())
        with torch.no_grad():
          log_var_extract.append(torch.exp(0.5 * log_var.cpu()).to(device))
          log_var_p_extract.append(torch.exp(0.5 * logvar_p.cpu()).to(device))
          mean_extract.append(means.cpu())
      
        loss, BCE, KLD, prop, kld_raw = loss_fn.forward(output.float(),
                              smiles_dec_output,
                              means, log_var, mu_p, logvar_p,
                              tgt, properties.float().squeeze(), tgt_z,
                              i)
        loss.backward()
        optim.step()
        optim2.step()

        batchloss += loss.item()
        kld_raw_batch += kld_raw.item()
        bce += BCE.item()
        kld += KLD.item()
        
    #Loss 값 추가
    loss = batchloss / len(train_dataloader)
    loss_arr.append(loss)

    # 토큰 복원
    original_tokens = reverse_one_hot_encoding(smiles_dec_output[50], dataset.vocab)
    predicted_tokens = reverse_one_hot_encoding(output[50], dataset.vocab)

    original_str = atomInSmiles.decode(' '.join(original_tokens))
    predicted_str = atomInSmiles.decode(' '.join(predicted_tokens))


    # 진행 바의 속성으로부터 필요한 값들 추출 (예시)
    elapsed = int(progress.format_dict.get("elapsed", 0))
    formatted_elap = str(datetime.timedelta(seconds=elapsed))
    rate = progress.format_dict.get("rate", None)
    sec_per_iter = 1 / rate if rate and rate != 0 else 0
    total = int(sec_per_iter * progress.total)
    formatted_total = str(datetime.timedelta(seconds=total))
    
    # 고정된 상태 정보를 업데이트 (Output 위젯에 출력)
    with status_out:
        clear_output(wait=True)
        print(f"🔹 Elapsed: {formatted_elap} | sec/iter: {sec_per_iter:.3f}s")
        print("🔹 Total time: ", formatted_total)
        print(f"🔹 Step: {i+1}/{progress.total}")
        print("🔹 loss: {:0.6f}".format(loss))
        print("[Posterior] sigma mean : {:0.6f}, ".format(torch.cat(log_var_extract).mean().item()),
              "sigma std : {:0.6f}".format(torch.cat(log_var_extract).std().mean()))
        print("[Prior]     sigma mean : {:0.6f}, ".format(torch.cat(log_var_p_extract).mean().item()),
              "sigma std : {:0.6f}".format(torch.cat(log_var_p_extract).std().mean()))
        print("raw KLD : ", kld_raw_batch / len(train_dataloader))
        print("Posterior μ per-dim std:", torch.cat(mean_extract,dim=0).view(-1, 64).std(axis=0).detach().cpu().numpy().mean())
        print("Posterior μ overall std:", torch.cat(mean_extract,dim=0).view(-1, 64).std().item())
        print("BCE : {:0.6f},".format(bce / 42 / len(train_dataloader)),
              "KLD : {:0.6f},".format(kld / 42 / 64 / len(train_dataloader)),
              "prop : {:0.6f},".format(prop))
        print(f"[Epoch {i}] Original : {original_str}")
        print(f"[Epoch {i}] Predict  : {predicted_str}")

Output()

Training:   0%|          | 0/1200 [00:00<?, ?it/s]

In [ ]:
model.eval()
prior.eval()
results = []
origin = []

properties_results=[]
properties_origin=[]
print(len(val_dataset))
with torch.no_grad():
    try:
        for (smiles_enc, smiles_dec_input, smiles_dec_output, properties) in val_dataloader:

            smiles_enc = smiles_enc.to(device)
            smiles_dec_input = smiles_dec_input.to(device)
            smiles_dec_output = smiles_dec_output.to(device)
            properties = properties.to(device)

            result, tgt, means, log_var, z = model(smiles_enc, smiles_dec_input, properties)

            results.append(result)
            origin.append(smiles_dec_output)

            properties_results.append(tgt)
            properties_origin.append(properties)
    except:
        for (smiles_dec_input, smiles_dec_output, properties) in val_dataloader_LSTM:

            smiles_dec_input = smiles_dec_input.to(device)
            smiles_dec_output = smiles_dec_output.to(device)
            properties = properties.to(device)

            result, tgt, means, log_var, z = model(smiles_dec_input, smiles_dec_output, properties)

            results.append(result)
            origin.append(smiles_dec_output)

            properties_results.append(tgt)
            properties_origin.append(properties)

results = torch.cat(results, dim=0)
origin = torch.cat(origin, dim=0)
results = nn.functional.softmax(results, dim=-1) 
argmax_indices = torch.argmax(results, dim=-1)
output = torch.nn.functional.one_hot(argmax_indices, num_classes=results.size(-1))

print(results.shape)
print(origin.shape)

from sklearn.metrics import mean_absolute_error
properties_origin=torch.cat(properties_origin,dim=0)
properties_results=torch.cat(properties_results,dim=0)
MAE_2 = mean_absolute_error(properties_origin.squeeze().cpu(), properties_results.squeeze().cpu())
print("MAE(properties) : ", MAE_2)


results_smiles = []
origin_smiles = []
for i in range(results.shape[0]):
    results_smiles.append(atomInSmiles.decode(' '.join(reverse_one_hot_encoding(results[i], dataset.vocab))))
    origin_smiles.append(atomInSmiles.decode(' '.join(reverse_one_hot_encoding(origin[i], dataset.vocab))))


1567
torch.Size([1567, 42, 167])
torch.Size([1567, 42])
MAE(properties) :  0.0635036418235403


In [ ]:
origin_smiles = [smiles.removesuffix("EOS").strip() for smiles in origin_smiles]
results_smiles = [smiles.removesuffix("EOS").strip() for smiles in results_smiles]


for i in range(len(results_smiles)):
    if(origin_smiles[i] != results_smiles[i]):
        print(i, "번째 다름!")
    print("real smiles      : ", origin_smiles[i])
    print("predicted smiles : ", results_smiles[i])


MAE = mean_absolute_error(origin.cpu(), torch.argmax(results.cpu(), dim=-1))
print("MAE : ", MAE)



real smiles      :  *OCC(C)N(CCC)CC(C)OC(*)=O
predicted smiles :  *OCC(C)N(CCC)CC(C)OC(*)=O
1 번째 다름!
real smiles      :  *OC(CC)C(NC(*)=O)C(=O)OCC
predicted smiles :  *OC(CC)C(CC(*)=O)C(=O)OCC
real smiles      :  *OCC(C)NC(=O)C(C)C(CC)OC(*)=O
predicted smiles :  *OCC(C)NC(=O)C(C)C(CC)OC(*)=O
real smiles      :  *NCCCCC(=O)NCC(CC)NC(*)=O
predicted smiles :  *NCCCCC(=O)NCC(CC)NC(*)=O
real smiles      :  *OCC(NC(*)=O)C(=O)NCCCC
predicted smiles :  *OCC(NC(*)=O)C(=O)NCCCC
real smiles      :  *OCC(=O)NCC(COC(*)=O)C(C)C
predicted smiles :  *OCC(=O)NCC(COC(*)=O)C(C)C
real smiles      :  *OC(C=O)C(O)=C(O)COC(*)=O
predicted smiles :  *OC(C=O)C(O)=C(O)COC(*)=O
real smiles      :  *OCC(C=C)NC(=O)CCCCOC(*)=O
predicted smiles :  *OCC(C=C)NC(=O)CCCCOC(*)=O
real smiles      :  *NCCCN(C)CC(CSC)OC(*)=O
predicted smiles :  *NCCCN(C)CC(CSC)OC(*)=O
9 번째 다름!
real smiles      :  *NCC(COC(C)(C)C)OC(*)=O
predicted smiles :  *NCC(COC(C)(C)O)OC(*)=O
real smiles      :  *OC(C)C(C)NC(CC)CCOC(*)=O
predicted smiles

In [ ]:
from rdkit import Chem, RDLogger
from rdkit.Chem import DataStructs, rdFingerprintGenerator
RDLogger.DisableLog('rdApp.error')

generator = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

def tanimoto_similarity(smiles1, smiles2):
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)
    fp1 = generator.GetFingerprint(mol1)
    fp2 = generator.GetFingerprint(mol2)
    return DataStructs.TanimotoSimilarity(fp1, fp2)

def is_valid(smiles):
    return Chem.MolFromSmiles(smiles) is not None

TS = 0.0
canbe = 0
notbe = 0

for sm, orig in zip(results_smiles, origin_smiles):
    if is_valid(sm) and is_valid(orig):
        sim = tanimoto_similarity(sm, orig)
        TS += sim
        canbe += 1
    else:
        notbe += 1

if canbe > 0:
    print("Tanimoto Similarity : ", TS / canbe)
else:
    print("No valid molecules to compare.")

print("가능한 분자 개수 :", canbe)
print("불가능한 분자 개수 :", notbe)
print("Valid fraction      :", canbe / len(results_smiles))


Tanimoto Similarity :  0.9605388401460709
가능한 분자 개수 : 1475
불가능한 분자 개수 : 92
Valid fraction      : 0.9412890874282067


In [ ]:
def save_weights(choice):
    if choice == "Trans_MHA":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256.pth")
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior.pth")        
        torch.save(prior.state_dict(), save_path)

    # Trans
    elif choice == "Trans":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256_no_mha.pth")
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior_no_mha.pth")        
        torch.save(prior.state_dict(), save_path)
    # LSTM
    elif choice == "LSTM":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM.pth")        
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_prior.pth")       
        torch.save(prior.state_dict(), save_path)


    # LSTM + MHA
    elif choice == "LSTM_MHA":
        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA.pth")        
        torch.save(model.state_dict(), save_path)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA_prior.pth")        
        torch.save(prior.state_dict(), save_path)

In [ ]:
save_weights(mode)